# Perfilamiento con time

In [2]:
import time

In [13]:
!pip install --quiet "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-1-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

You are using pip version 19.0.3, however version 21.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [84]:
#!pip uninstall mex -y

Uninstalling mex-0.1:
  Successfully uninstalled mex-0.1


In [15]:
import copy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pprint
from scipy.optimize import linprog
from pytest import approx

from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, constrain, add_obj, obj, maxz, minz

## Objetivo

In [16]:
c_max_obj_1 = [-3, -5]
A_max_obj_1 = [[1, 0], [0, 2], [3, 2]]
b_max_obj_1 = [4, 12, 18]

In [17]:
max_obj_1 = -1*linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).fun

In [18]:
coeff_obj_1 = linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).x

## Aproximación

In [19]:
n_var_approx_1 = 2
n_cons_approx_1 = 3

In [20]:
start_time = time.time()
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)
end_time = time.time()
secs = end_time-start_time
print("create_matrix tomó",secs,"segundos" )

create_matrix tomó 0.0002834796905517578 segundos


In [21]:
start_time = time.time()
constrain(matrix_max_approx_1,'1,0,L,4')
end_time = time.time()
secs = end_time-start_time
print("constrain tomó",secs,"segundos" )

constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')

constrain tomó 0.0005183219909667969 segundos


In [22]:
start_time = time.time()
obj(matrix_max_approx_1,'3,5,0')
end_time = time.time()
secs = end_time-start_time
print("obj tomó",secs,"segundos" )

obj tomó 0.0004169940948486328 segundos


In [28]:
start_time = time.time()
problem_approx_1 = maxz(matrix_max_approx_1)
end_time = time.time()
secs = end_time-start_time
print("maxz tomó",secs,"segundos" )

maxz tomó 0.001978158950805664 segundos


In [30]:
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))

In [34]:
# Todo junto
start_time = time.time()
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)
constrain(matrix_max_approx_1,'1,0,L,4')
constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')
obj(matrix_max_approx_1,'3,5,0')
problem_approx_1 = maxz(matrix_max_approx_1)
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))
end_time = time.time()
secs = end_time-start_time
print("Todo el proceso tomó",secs,"segundos" )

Todo el proceso tomó 0.003371000289916992 segundos


## Comprobación

In [31]:
assert max_obj_1 == approx(max_approx_1), "El valor aproximado es incorrecto"
assert coeff_obj_1 == approx(coeff_approx_1), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_1)
print("El valor aproximado obtenido con mex es: ", max_approx_1)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_1)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_1)

El valor objetivo obtenido con scipy es:  35.999999978433735
El valor aproximado obtenido con mex es:  36.0
Los coeficientes objetivos obtenidos con scipy son:  [2. 6.]
Los coeficientes aproximados obtenidos con mex son:  [2. 6.]


# Con comandos magic

Sólo probaremos con maxz que es la función que más se tardó en el proceso anterior

In [36]:
%time maxz(matrix_max_approx_1)

CPU times: user 653 µs, sys: 222 µs, total: 875 µs
Wall time: 888 µs


{'x1': 2.0, 'x2': 6.0, 'max': 36.0}

In [35]:
%timeit -n 5 -r 10 maxz(matrix_max_approx_1)

181 µs ± 54.3 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


# Con cProfile

In [1]:
import cProfile

In [37]:
cprof = cProfile.Profile()
cprof.enable()
problem_approx_1 = maxz(matrix_max_approx_1)
cprof.disable()
cprof.print_stats(sort='cumtime')

         151 function calls in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:3386(run_code)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <ipython-input-37-4199b605bc38>:3(<module>)
        1    0.000    0.000    0.000    0.000 problem_definition.py:178(maxz)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:174(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:92(find_negative_row)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        7    0.000    0.000    0.000    0.000 simplex_networks.py:47(pivots_row)
       16    0.000    0.000    0.000    

## con comando magic

In [38]:
 %prun -s cumulative problem_approx_1 = maxz(matrix_max_approx_1)

In [41]:
cprof.dump_stats("maxz_stats")

In [42]:
import pstats

In [43]:
p_max_stats = pstats.Stats("maxz_stats")
print(p_max_stats.sort_stats("cumulative").print_stats(10))

Wed May  5 18:15:34 2021    maxz_stats

         151 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 39 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3386(run_code)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <ipython-input-37-4199b605bc38>:3(<module>)
        1    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py:178(maxz)
        4    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 /home/user/.pyenv/versi

In [55]:
print(p_max_stats.sort_stats("cumulative").print_stats("simplex|lambda|maxz"))

Wed May  5 18:15:34 2021    maxz_stats

         151 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 39 to 7 due to restriction <'simplex|lambda|maxz'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 problem_definition.py:178(maxz)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:174(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:92(find_negative_row)
        7    0.000    0.000    0.000    0.000 simplex_networks.py:47(pivots_row)
        1    0.000    0.000    0.000    0.000 simplex_networks.py:26(pivots_col)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:117(<lambda>)




In [54]:
print(p_max_stats.strip_dirs().sort_stats("cumulative").print_stats("lambda|simplex|maxz"))

Wed May  5 18:15:34 2021    maxz_stats

         151 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 39 to 7 due to restriction <'lambda|simplex|maxz'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 problem_definition.py:178(maxz)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:174(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:92(find_negative_row)
        7    0.000    0.000    0.000    0.000 simplex_networks.py:47(pivots_row)
        1    0.000    0.000    0.000    0.000 simplex_networks.py:26(pivots_col)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:117(<lambda>)




In [48]:
# El número de veces que se llamaron a funciones primitivas o built in: 
print(p_max_stats.prim_calls)

151


In [50]:
# qué funciones llaman a otras
p_max_stats.strip_dirs().sort_stats("cumulative").print_callers()

   Ordered by: cumulative time

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3386(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       2    0.000    0.000  interactiveshell.py:3386(run_code)
<ipython-input-37-4199b605bc38>:3(<module>)                              <-       1    0.000    0.000  {built-in method builtins.exec}
problem_definition.py:178(maxz)                                          <-       1    0.000    0.000  <ipython-input-37-4199b605bc38>:3(<module>)
simplex_networks.py:146(find_pivot_row)                                  <-       4    0.000    0.000  problem_definition.py:178(maxz)
simplex_networks.py:174(pivot)                                           <-       2    0.000    0.000  problem_definition.py:178(maxz)
simplex_n

In [51]:
p_max_stats.strip_dirs().sort_stats("cumulative").print_callers()

   Ordered by: cumulative time

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3386(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       2    0.000    0.000  interactiveshell.py:3386(run_code)
<ipython-input-37-4199b605bc38>:3(<module>)                              <-       1    0.000    0.000  {built-in method builtins.exec}
problem_definition.py:178(maxz)                                          <-       1    0.000    0.000  <ipython-input-37-4199b605bc38>:3(<module>)
simplex_networks.py:146(find_pivot_row)                                  <-       4    0.000    0.000  problem_definition.py:178(maxz)
simplex_networks.py:174(pivot)                                           <-       2    0.000    0.000  problem_definition.py:178(maxz)
simplex_n

In [52]:
# Análogamente podemos establecer los filtros usados en print_stats
p_max_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 39 to 10 due to restriction <10>

Function                                     was called by...
                                                 ncalls  tottime  cumtime
interactiveshell.py:3386(run_code)           <- 
{built-in method builtins.exec}              <-       2    0.000    0.000  interactiveshell.py:3386(run_code)
<ipython-input-37-4199b605bc38>:3(<module>)  <-       1    0.000    0.000  {built-in method builtins.exec}
problem_definition.py:178(maxz)              <-       1    0.000    0.000  <ipython-input-37-4199b605bc38>:3(<module>)
simplex_networks.py:146(find_pivot_row)      <-       4    0.000    0.000  problem_definition.py:178(maxz)
simplex_networks.py:174(pivot)               <-       2    0.000    0.000  problem_definition.py:178(maxz)
simplex_networks.py:92(find_negative_row)    <-       4    0.000    0.000  simplex_networks.py:146(find_pivot_row)
codeop.py:132(__call__)                      <- 
{built-in metho

In [53]:
p_max_stats.strip_dirs().sort_stats("cumulative").print_callees("maxz|lambda")

   Ordered by: cumulative time
   List reduced from 39 to 2 due to restriction <'maxz|lambda'>

Function                           called...
                                       ncalls  tottime  cumtime
problem_definition.py:178(maxz)    ->       2    0.000    0.000  <__array_function__ internals>:2(where)
                                            2    0.000    0.000  general.py:21(gen_var)
                                            1    0.000    0.000  simplex_networks.py:26(pivots_col)
                                            3    0.000    0.000  simplex_networks.py:47(pivots_row)
                                            4    0.000    0.000  simplex_networks.py:146(find_pivot_row)
                                            2    0.000    0.000  simplex_networks.py:174(pivot)
                                            2    0.000    0.000  {built-in method builtins.len}
                                            2    0.000    0.000  {built-in method builtins.max}
         

# Con line profiler

In [59]:
!pip install line_profiler --quiet

You are using pip version 19.0.3, however version 21.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [60]:
import line_profiler

In [61]:
line_prof = line_profiler.LineProfiler()

print(line_prof(maxz)(matrix_max_approx_1))

{'x1': 2.0, 'x2': 6.0, 'max': 36.0}


In [62]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.001312 s
File: /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py
Function: maxz at line 178

Line #      Hits         Time  Per Hit   % Time  Line Contents
   178                                           def maxz(matrix):
   179                                               """
   180                                               Creates maximization function. Determines if 1 extra pivot is required, locates the pivot element,
   181                                               pivots about it and continues the process until all negative elements have been removed from
   182                                               the last column and row.
   183                                               
   184                                               Args:
   185                                               
   186                                                   matrix (numpy array): 

## Podemos agregar todas las funciones en el lineprofiler

In [69]:
# Todo junto
line_prof_all = line_profiler.LineProfiler()

line_prof_all.add_function(create_matrix)
matrix_max_approx_1 = line_prof_all(create_matrix)(n_var_approx_1,n_cons_approx_1)
line_prof_all.add_function(constrain)
line_prof_all(constrain)(matrix_max_approx_1,'1,0,L,4')
line_prof_all(constrain)(matrix_max_approx_1,'0,2,L,12')
line_prof_all(constrain)(matrix_max_approx_1,'3,2,L,18')
line_prof_all.add_function(obj)
line_prof_all(obj)(matrix_max_approx_1,'3,5,0')
line_prof_all.add_function(maxz)          
problem_approx_1 = line_prof_all(maxz)(matrix_max_approx_1)

In [70]:
print(line_prof_all.print_stats())

Timer unit: 1e-06 s

Total time: 0.001405 s
File: /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py
Function: constrain at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                           def constrain(matrix, eq):
    39                                               """
    40                                               Adds constraints to the problem.
    41                                               
    42                                               Args:
    43                                               
    44                                                   matrix (numpy array): matrix defined with :mod:`create_matrix`.
    45                                                   
    46                                                   eq (string): coefficients of constraints expressions. Use **L** for *less than*, **G** for *greater than*, **E** for *eq

## Utilizando kernprof

In [ ]:
!pip uninstall mex -y

In [85]:
!pip install --quiet pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

You are using pip version 19.0.3, however version 21.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [90]:
import os
os.chdir("..")
current_path = os.getcwd()

In [93]:
## Tuve que hacer sudo apt-install python3-line-profiler -y en mi linux (hay que hacerlo en la imagen)

### Para maxz

In [104]:
path = os.path.join(current_path,'src','profiling','tiempo','max_line_profiler.py')

In [108]:
%%bash -s $path
kernprof -l -v "$1"

Wrote profile results to max_line_profiler.py.lprof
Timer unit: 1e-06 s

Total time: 0.000184 s
File: /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py
Function: add_cons at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def add_cons(matrix):
    10                                               """
    11                                               Checks if 1 extra constraint can be added to the matrix, this means that there are at least two rows of all
    12                                               0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.
    13                                               
    14                                               Args:
    15                                               
    16    

### Para minz

In [110]:
path = os.path.join(current_path,'src','profiling','tiempo','min_line_profiler.py')

In [111]:
%%bash -s $path
kernprof -l -v "$1"

Wrote profile results to min_line_profiler.py.lprof
Timer unit: 1e-06 s

Total time: 7.7e-05 s
File: /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py
Function: add_cons at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def add_cons(matrix):
    10                                               """
    11                                               Checks if 1 extra constraint can be added to the matrix, this means that there are at least two rows of all
    12                                               0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.
    13                                               
    14                                               Args:
    15                                               
    16     

### Para la red con problema de minimización

In [113]:
path = os.path.join(current_path,'src','profiling','tiempo','net_line_profiler.py')

In [114]:
%%bash -s $path
kernprof -l -v "$1"

Wrote profile results to net_line_profiler.py.lprof
Timer unit: 1e-06 s

Total time: 0.001676 s
File: /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py
Function: add_cons at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def add_cons(matrix):
    10                                               """
    11                                               Checks if 1 extra constraint can be added to the matrix, this means that there are at least two rows of all
    12                                               0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.
    13                                               
    14                                               Args:
    15                                               
    16    

/home/user/Documents/Caro_MCD/Optimizacion/practica-2-segunda-parte-caroacostatovany/src/profiling/tiempo/net_line_profiler.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  net_obj = linprog(c_net_obj, A_ub=A_ub_net_obj, b_ub=b_ub_net_obj, A_eq=A_eq_net_obj, b_eq=b_eq_net_obj).fun
/home/user/Documents/Caro_MCD/Optimizacion/practica-2-segunda-parte-caroacostatovany/src/profiling/tiempo/net_line_profiler.py:23: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  net_coeff_obj = linprog(c_net_obj, A_ub=A_ub_net_obj, b_ub=b_ub_net_obj, A_eq=A_eq_net_obj, b_eq=b_eq_net_obj).x
